Copyright (c) 2024 Agnese Re <agnesere43@gmail.com>\
GitHub account: https://github.com/AgneseRe

In [846]:
# Import libraries
import pandas as pd
import numpy as np
import copy
import matplotlib.pyplot as plt

from icecream import ic
from itertools import combinations
from geopy.distance import geodesic
from dataclasses import dataclass

## Read csv and compute distance matrix
Read a comma-separated values (csv) file into DataFrame using *read_csv* function of **pandas** library. The file does not contain any header, the first line already contains the name and geographical coordinates of a city. Columns names are passed explicitly to *names* parameter. Using the *geodesic* function of **geopy** library, the geodesic distance in kilometers between each pair of cities is computed.


In [847]:
CITIES = pd.read_csv("cities/china.csv", sep = ",", header = None, names = ['Name', 'Lat', 'Long'])
DIST_MATRIX = np.zeros((len(CITIES), len(CITIES)))
for c1, c2 in combinations(CITIES.itertuples(), 2):
    DIST_MATRIX[c1.Index, c2.Index] = DIST_MATRIX[c2.Index, c1.Index] = geodesic(
        (c1.Lat, c1.Long), (c2.Lat, c2.Long)
    ).km

In [848]:
CITIES.head()   # first 5 rows of the CITIES dataframe

,Name,Lat,Long
0,Acheng,45.540000,126.960000
1,Aksu,41.150000,80.250000
2,Alaer,40.515556,81.263611
3,Altay,47.840000,88.130000
4,Anbu,23.460000,116.680000


In [849]:
CITIES_NAMES, CITY_INDEXES = map(np.array, zip(*[(city['Name'], i) for i, city in CITIES.iterrows()]))

## Data Class and fitness function definition
The data class *Individual* is defined. The *genome* attribute holds an individual's genome. The *fitness* attribute quantifies how well the individual performs with respect to the objective function. Initially, it is set to *None*. The fitness score will be calculate later, after the genome is evaluated.  
*e.g.* If the number of cities for TSP is equal to 8, a possible *Individual* object is ```Individual(genome=array([7, 3, 0, 2, 1, 6, 4, 5]), fitness=np.float64(-2306.6849123742595))```.

In [850]:
@dataclass
class Individual:
    genome: np.ndarray
    fitness: float = None

def calculate_fitness(individual: Individual) -> float:
    tot_cost = 0.0
    for c1, c2 in zip(individual.genome, np.concatenate((individual.genome[1:], individual.genome[:1]))):
        tot_cost += DIST_MATRIX[c1, c2]
    return -tot_cost

## Greedy Approach
Start from city 0 and move, from time to time, to the *nearest* city. It's an algorithm that leads to immediate cost gain, but becomes awful in the process. It is a relatively fast approach (*e.g.* 8 steps in case of 8 cities), but it leads to an approximate solution.

In [ ]:
# At the beginning, no city is visited
visited = np.full(len(CITIES), False)

# Set the starting point and set it to visited
city = 0
visited[city] = True
dist = DIST_MATRIX.copy()

# List to store results
tsp = []
tsp.append(int(city))

while not np.all(visited):
    # Find the minimum cost and the corresponding city among those not yet visited
    closest_cost = np.min([dist[city, adj] for adj in range(len(CITIES)) if not visited[adj] and adj != city])
    closest = np.where(dist[city] == closest_cost)[0][0]    # e.g. (array([7]),) -> tuple first element, array first element
    # Set the found city to visited and add it to the tsp list
    visited[closest] = True
    tsp.append(int(closest))
    # Update city
    city = closest

# Print result
best_path = Individual(tsp)
best_path.fitness = calculate_fitness(best_path)
print(f"{len(CITIES)} cities, shortest tour is {-best_path.fitness:.2f} km")
print([str(CITIES_NAMES[index]) for index in tsp])

## Evolutionary Algorithm

A population is made of *Individual*s, where each of them represents a unique ordered sequence of cities. The initial population is created using a *greedy* approach. Instead of starting with purely random permutations, each individual begins from a randomly selected city and follows a "greedy neighbor" heuristic to build the path, choosing one of the closest three cities at each step. The choice of starting from an almost *random* population helps provide a wide search base for the **evolutionary** algorithm, allowing it to explore and optimize solutions more effectively.

In [851]:
POPULATION_SIZE = 100
population = []
rng = np.random.default_rng()

In [852]:
def greedy_neighbor(start_city, k = 3):
    unvisited = set(CITY_INDEXES)
    unvisited.remove(start_city)
    current_city = start_city
    path = [int(current_city)]
    
    while unvisited:
        # Choose a random city from the closest three (element of randomness)
        neighbors = sorted(unvisited, key=lambda city: DIST_MATRIX[current_city][city])[:k]
        next_city = rng.choice(neighbors)

        path.append(int(next_city))
        unvisited.remove(next_city)
        current_city = next_city
    
    new_individual = Individual(np.array(path))
    new_individual.fitness = calculate_fitness(new_individual)
    
    return new_individual

### Parents selection

In [853]:
def parent_selection(population: np.ndarray) -> Individual:
    candidates = sorted(np.random.choice(population, 2), key = lambda i: i.fitness, reverse=True)
    return candidates[0]

### Mutation and Cross-over

A new offspring is constructed mutating one selected parent or crossing over two of them. The *displacement mutation* and the *inver-over crossover* were found to be the best strategies for obtaining good results.

In [854]:
# Two possible types of mutation
def swap_mutation(individual: Individual) -> Individual:
    new_individual = copy.deepcopy(individual)
    i1, i2 = np.random.choice(len(individual.genome), 2, replace = False)
    new_individual.genome[i1], new_individual.genome[i2] = new_individual.genome[i2], new_individual.genome[i1]
    new_individual.fitness = calculate_fitness(new_individual)
    return new_individual

def displacement_mutation(individual: Individual) -> Individual:
    genome = individual.genome
    start = np.random.randint(0, len(genome) - 2)   # cut point 1
    end = np.random.randint(start + 1, len(genome) - 1) # cut point 2
    segment = genome[start: end+1]
    genome = np.concatenate((genome[:start], genome[end+1:]))
    random_location = np.random.randint(0, len(genome))
    new_genome = np.concatenate((genome[:random_location], segment, genome[random_location:]))
    new_individual = Individual(new_genome)
    new_individual.fitness = calculate_fitness(new_individual)
    return new_individual

# Two possible types of cross-over
def order_crossover(parent1: Individual, parent2: Individual):
    parent1, parent2 = Individual(parent1.genome, parent1.fitness), Individual(parent2.genome, parent2.fitness)
    size = len(parent1.genome)

    child = np.full(size, -1)
    start, end = sorted(np.random.choice(size, 2, replace=False))
    child[start:end+1] = parent1.genome[start:end+1]
    remaining_cities = [city for city in parent2.genome if city not in child]
    
    j = 0
    for i in range(size):
        if child[i] == -1:  # Se è un segnaposto (-1)
            child[i] = remaining_cities[j]
            j += 1

    new_individual = Individual(child)
    new_individual.fitness = calculate_fitness(new_individual)
    
    return new_individual

def inver_over(parent1: Individual, parent2: Individual) -> Individual:
    genome1, genome2 = parent1.genome, parent2.genome

    # select one random gene from the first parent
    index1 = np.random.randint(0, len(genome1))
    gene1 = genome1[index1]

    # select edge from the second parent (only one trait)
    index2 = np.where(genome2 == gene1)[0][0]
    gene2 = genome2[index2+1] if index2 != len(genome2)-1 else genome2[0]   # cyclic genotype

    # generate offspring
    flip_parent1 = np.concatenate((genome1[index1:], genome1[:index1]))
    index_gene2_in_flip_parent1 = np.where(flip_parent1 == gene2)[0][0]
    sequence = flip_parent1[1:index_gene2_in_flip_parent1]
    offspring_genome = np.concatenate((np.array([gene1]), np.array([gene2]), sequence[::-1], flip_parent1[index_gene2_in_flip_parent1+1:]))
    
    offspring = Individual(offspring_genome)
    offspring.fitness = calculate_fitness(offspring)

    return offspring

### Hill climbing and Simulated Annealing
Local search optimization techniques have been integrated into the genetic algorithm. The *hill climbing* technique was found to be useful for achieving optimal results.

In [855]:
MAX_ATTEMPTS = 20
INITIAL_TEMP = 100
COOLING_RATE = 0.95

def simulated_annealing(individual: Individual) -> Individual:
    improved_individual = copy.deepcopy(individual)
    temperature = INITIAL_TEMP
    for _ in range(MAX_ATTEMPTS):
        neighbor = displacement_mutation(improved_individual)
        delta_fitness = neighbor.fitness - improved_individual.fitness
        if delta_fitness > 0 or np.random.random() < np.exp(delta_fitness/temperature):
            improved_individual = neighbor
        temperature *= COOLING_RATE
    return improved_individual

def hill_climbing(individual: Individual) -> Individual:
    improved_individual = copy.deepcopy(individual)
    for _ in range(MAX_ATTEMPTS):
        neighbor = displacement_mutation(improved_individual)
        delta_fitness = neighbor.fitness - individual.fitness
        if delta_fitness > 0:
            improved_individual = neighbor
    return improved_individual

In [857]:
# Initial population
for _ in range(POPULATION_SIZE):
       start_city = int(rng.choice(CITY_INDEXES))
       population.append(greedy_neighbor(start_city))
# print(sorted([i.fitness for i in population], reverse=True))

### Process

In [ ]:
CROSSOVER_PROBABILITY = .40
OFFSPRING_SIZE = POPULATION_SIZE//5
NUM_GENERATIONS = 100_000

for _ in range(NUM_GENERATIONS):
    population.sort(key = lambda i: i.fitness, reverse=True)
    offsprings = []
    
    while len(offsprings) < OFFSPRING_SIZE:
        xover_probability = np.random.random()
        if xover_probability < CROSSOVER_PROBABILITY:
            parent1 = parent_selection(population)
            parent2 = parent_selection(population)
            offspring = inver_over(parent1, parent2)
        else:   # no crossover
            parent1 = parent_selection(population)
            offspring = displacement_mutation(parent1)

        offsprings.append(hill_climbing(offspring))
    population.extend(offsprings)
    population.sort(key=lambda i: i.fitness, reverse=True)
    population = population[:POPULATION_SIZE]

population.sort(key = lambda i: i.fitness, reverse = True)
shortest_path = [str(CITIES_NAMES[index]) for index in population[0].genome]
print(f"{len(CITIES)} cities, shortest tour is {-population[0].fitness:.2f} km")
print(shortest_path)

## Plotting

In [ ]:
lat_shortest = [CITIES.iloc[index]['Lat'] for index in population[0].genome]
long_shortest = [CITIES.iloc[index]['Long'] for index in population[0].genome]
lat_shortest.append(lat_shortest[0])
long_shortest.append(long_shortest[0])

fig, ax = plt.subplots()
ax.plot(lat_shortest, long_shortest, linestyle="-.", color="blue", label='Best Route', linewidth=1)
ax.scatter(lat_shortest, long_shortest, color='red', marker='o', label='Cities', s=20)
plt.legend()

plt.title(label="TSP Best Route Using GA", fontsize=15, color="k")
str_params = f'\n{NUM_GENERATIONS} Generations\n{POPULATION_SIZE} Population Size\n{CROSSOVER_PROBABILITY:.2f} Cross-over'
plt.suptitle("Total Distance Travelled: " + 
             str(round(population[0].fitness, 2))[1:] + " km" + 
             str_params, fontsize=15, y = 1.047)

fig.set_size_inches(8, 8)  
plt.tight_layout()  
plt.savefig('plots/tsp_best_route_china_100_000.png', dpi=300, bbox_inches='tight')
plt.show()